# Narzędzie do importu do Excel punktów z Mapy Google na Bieg Nocny.
Żeby uruchomić program musisz kliknąć symbol startu obok komórek z programem. Wykonaj je **PO KOLEI**.

In [6]:
#@title Inicjalizacja
!pip install fastkml -q
from google.colab import files
import io
from fastkml import kml
import pandas as pd
import numpy as np

In [9]:
#@title Wgraj plik KML
#@markdown Żeby pobrać swoją Mapę Google jako .kml:
#@markdown - wybierz przycisk z symbolem trzech kropek -> pobierz KML
#@markdown - **zaznacz opcje Export do pliku KML zamiast KMZ**
#@markdown - uruchom komórkę i wgraj plik

#@markdown Wybierz id twojej mapy (zwykle 0)
uploaded = files.upload()

if not uploaded:
    raise ValueError("Please upload a file before continuing.")

filename = list(uploaded.keys())[0]

if not filename.lower().endswith('.kml'):
    raise ValueError("The uploaded file is not a KML file. Please upload a .kml file.")

try:
    kml_obj = kml.KML.parse(filename)
except Exception as e:
    raise ValueError("Could not read KML. Please check your file.") from e

map_id = 0 #@param {"type":"integer"}
map_to_import = kml_obj.features[0].features[0].features
print(f"Wczytano mapę {map_id} z pliku {filename} zawierającą {len(map_to_import)} punktów.")

Saving trasa HS 2025.kml to trasa HS 2025.kml
Wczytano mapę 0 z pliku trasa HS 2025.kml zawierającą 120 punktów.


In [10]:
#@title Wczytaj plik Excel do wypełnienia
#@markdown Ustaw nazwę arkusza do którego chcesz importować.

#@markdown Następnie uruchom komórkę i wgraj plik .xlsx do wypełnienia.

route_sheet_name = "Trasa R" #@param {"type":"string"}

uploaded = files.upload()

if not uploaded:
    raise ValueError("Please upload a file before continuing.")

excel_filename = list(uploaded.keys())[0]

if not excel_filename.lower().endswith('.xlsx'):
    raise ValueError("The uploaded file is not a XLSX file. Please upload a .xlsx file.")

try:
    output_df = pd.read_excel(excel_filename, sheet_name=route_sheet_name, engine='openpyxl')
except Exception as e:
    raise ValueError("Could not read XLSX. Please check your file.") from e


output_df.head()

Saving Punkty puste.xlsx to Punkty puste (2).xlsx


,NR Punktu,Kod,Nazwa Punktu,Współrzędna N,Współrzędna E,ID Pytania,Odpowiedź


In [11]:
#@title Wypełnianie tabeli
#@markdown Po uruchomieniu komórka wypełni tabelę punktami z mapy.

#@markdown Wybierz pierwszą literę kodu punktu - powninna być to inna litra dla każdej z tras w danej edycji.

#@markdown Ustaw czy chcesz żeby program dodał id pytań (po kolei) oraz czy ma dodać losowe odpowiedzi (ABCD)

point_code_first_letter = 'B' # @param {"type":"string"}
point_codes = [f"{point_code_first_letter}{chr(65 + i)}{chr(65 + j)}" for i in range(26) for j in range(26)]
np.random.shuffle(point_codes)
add_question_ids = True #@param {"type":"boolean"}
add_random_answers = True #@param {"type":"boolean"}

for i, placemark in enumerate(map_to_import):
    row = dict()
    row['NR Punktu'] = i + 1
    row['Kod'] = point_codes[i]
    row['Nazwa Punktu'] = placemark.name
    row['Współrzędna N'] = placemark.geometry.coords[0][1]
    row['Współrzędna E'] = placemark.geometry.coords[0][0]
    if add_question_ids:
        row['ID Pytania'] = f"P{i+1}"
    if add_random_answers:
        row['Odpowiedź'] = np.random.choice(['A', 'B', 'C', 'D'])

    output_df.loc[i] = row
output_df.head()


,NR Punktu,Kod,Nazwa Punktu,Współrzędna N,Współrzędna E,ID Pytania,Odpowiedź
0,1,BQN,Pomnik Zygi Latarnika,52.403706,16.942551,P1,C
1,2,BIK,Skwer Obrońców Ukrainy 2022,52.400832,16.947305,P2,C
2,3,BGO,"Rzeźba ""Lucie"" Davida Mesguicha",52.398481,16.953886,P3,A
3,4,BWS,Polanka,52.396944,16.962239,P4,B
4,5,BKE,Żabka,52.392909,16.964364,P5,A


In [13]:
#@title Zapisz i pobierz
filename = "Punkty.xlsx" #@param {type:"string"}
output_df.to_excel(filename, index=False)
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>